# Aula 4 - Parte prática - Actor-Critic (A2C)

## Introdução

Nesse quarto notebook ...

$$
\nabla_\theta J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta} \left[ \sum_{t=0}^{T-1} \nabla_\theta \log \pi_\theta(\mathbf{a}_t|\mathbf{s}_t) \left ( \left( \sum_{k=t}^{T-1} r_{k} \right)  - b(\mathbf{s}_{t}) \right ) \right]
$$


### Objetivos:

- ambientes vetorizados (gym.vector.make)
- retornos descontados
- 2-head net / joint
- entropy bonus
- n-step return (bootstrap)
- clip gradient by norm
- learning_rate linear scheduler (decay)


In [ ]:
import logging

import gym
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

from utils.agent import RLAgent
from utils.memory import OnPolicyReplay
from utils.networks import build_actor_critic_network
import utils.runner
from utils.viz import *


tf.get_logger().setLevel("ERROR")     # ignore TensorFlow warnings
gym.logger.set_level(logging.ERROR)   # ignore OpenAI Gym warnings

In [ ]:
class A2C(RLAgent):
    
    def __init__(self, obs_space, action_space, config):
        super().__init__(obs_space, action_space, config)
        
        self.memory = OnPolicyReplay()
        self.actor_critic = build_actor_critic_network(obs_space, action_space, config["actor_critic_net"])

        self.optimizer = tf.keras.optimizers.RMSprop(
            learning_rate=tf.keras.optimizers.schedules.PolynomialDecay(
                0.00083,
                decay_steps=config["total_timesteps"] / config["train_batch_size"],
                end_learning_rate=1e-4,
                power=1.0
            ))

    def act(self, obs):
        """
        Escolhe uma ação para ser tomada dada uma observação do ambiente.
        
        Args: 
            obs: observação do ambiente.
        
        Return:
            action: ação válida dentro do espaço de ações.
        """
        return self._act(obs).numpy()
        
    @tf.function
    def _act(self, obs):
        action_dist, _ = self.actor_critic(obs)
        return action_dist.sample()
    
    def observe(self, obs, action, reward, next_obs, done):
        """
        Registra na memória do agente uma transição do ambiente.

        Args:
            obs:            observação do ambiente antes da execução da ação.
            action:         ação escolhida pelo agente.
            reward (float): escalar indicando a recompensa obtida após a execução da ação.
            next_obs:       nova observação recebida do ambiente após a execução da ação.
            done (bool):    True se a nova observação corresponde a um estado terminal, False caso contrário.

        Return:
            None
        """
        self.memory.update(obs, action, reward, next_obs, done)

    def learn(self):
        """
        Método de treinamento do agente. A partir das experiências de sua memória,
        o agente aprende um novo comportamento.

        Args: 
            None

        Return:
            None
        """
        if self.memory.batch_size < self.config["train_batch_size"]:
            return
        
        batch = self.memory.sample()
        weights = self.actor_critic.trainable_weights

        with tf.GradientTape() as tape:
            policy_loss, vf_loss, entropy_loss = self._joint_loss_fn(batch)
            loss = policy_loss + self.config["vf_loss_coeff"] * tf.cast(vf_loss, tf.float32) - self.config["entropy_coeff"] * entropy_loss
            gradients = tape.gradient(loss, weights)
    
        gradients = tuple(tf.clip_by_norm(grad, clip_norm=0.5) for grad in gradients)
        self.optimizer.apply_gradients(zip(gradients, weights))
      
        return {
            "policy_loss": policy_loss.numpy(),
            "vf_loss": vf_loss.numpy(),
            "entropy_loss": entropy_loss.numpy()
        }

    def _joint_loss_fn(self, batch):
        """
        Calcula a função loss do policy gradients para um `batch` de transições.
        
        Um `batch` agrega arrays n-dimensionais. Cada array (e.g., batch["states"],
        batch["actions"], batch["rewards"]) tem como primeiras duas dimensões o número
        de passos dados no ambiente vetorizado e o número de ambientes em paralelo. 
        Por exemplo, batch["states"][t][k] devolve um array correspondendo ao estado 
        no passo t devolvido pelo k-ésimo ambiente.

        Args:
            batch (Dict[str, np.ndarray]): dicionário para acesso às matrizes de 
                estados, ações, recompensas, próximos estados e flags de terminação. 
        
        Return:
            loss (tf.Tensor): surrogate loss conjunta da política, função valor e
                bônus de entropia.
        """
        vf_criterion = tf.keras.losses.MeanSquaredError()

        states = batch["states"]
        actions = batch["actions"]
        rewards = batch["rewards"]
        next_states = batch["next_states"]
        dones = batch["dones"]

        n_steps = len(states)
        gamma = self.config["gamma"]
        lam = self.config["lambda"]
        

        action_dists, values = self.actor_critic(states)
        _, last_value = self.actor_critic(next_states[-1:])
        
        values = tf.squeeze(tf.concat([values, last_value], axis=0))
        values, next_values = values[:-1], values[1:]

        deltas = rewards + gamma * (1 - dones) * next_values - values

        returns = np.empty_like(rewards)
        advantages = np.empty_like(rewards)

        returns[-1] = rewards[-1] + gamma * (1 - dones[-1]) * next_values[-1]
        advantages[-1] = deltas[-1]

        for t in reversed(range(n_steps - 1)):
            returns[t] = rewards[t] + gamma * (1 - dones[t]) * returns[t+1]
            advantages[t] = deltas[t] + (gamma * lam) * (1 - dones[t]) * advantages[t+1]

        log_probs = action_dists.log_prob(actions)

        policy_loss = - tf.reduce_sum(log_probs * tf.stop_gradient(advantages.astype("f")))
        vf_loss = vf_criterion(values, tf.stop_gradient(returns.astype("f")))
        entropy_loss = tf.reduce_mean(action_dists.entropy())

        return policy_loss, vf_loss, entropy_loss


In [ ]:
total_timesteps = 500_000

num_envs = 8
env = gym.vector.make("LunarLander-v2", num_envs=num_envs, asynchronous=True)

config = {
    "actor_critic_net": {
        "hidden_layers": [64, 64],
        "activation": "tanh"
    },
    "optimizer": {
        "class_name": "RMSprop",
        "config": {
            "learning_rate": 8e-4,
            "rho": 0.99
        }
    },
    "total_timesteps": total_timesteps,
    "train_batch_size": 40,
    "gamma": 0.995,
    "lambda": 1.0,
    "vf_loss_coeff": 0.25,
    "entropy_coeff": 1e-5
}

In [ ]:
agent = A2C(env.single_observation_space, env.single_action_space, config)

tf.keras.utils.plot_model(agent.actor_critic, show_shapes=True)

In [ ]:
timesteps, total_rewards, avg_total_rewards = utils.runner.train(agent, env, total_timesteps)

In [ ]:
env = gym.make("LunarLander-v2")

for episode in range(20):
    obs = env.reset()    
    done = False
    total_reward = 0.0
    while not done:
        action = agent.act(obs[None,:])[0]
        next_obs, reward, done, _ = env.step(action)
        total_reward += reward
        env.render()
        obs = next_obs
    env.close()
    
    print(f"episode = {episode}, total_reward={total_reward}")